In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *

rt = RACETrack()

df = pd.DataFrame({
    'f':[10,     11,    10,    20,    30,   20,   10],
    'c':['a',    'b',   'c',   'a',   'b',  'a',  'c'],
    'd':['x',    'x',   'y',   'y',   'z',  'z',  'z']
})

_combos_ = list(df.columns)
_combos_.append(None)

In [ ]:
def countBySet(df, count_by):
    return count_by != None and df[count_by].dtypes != np.int64   and \
                                df[count_by].dtypes != np.int32   and \
                                df[count_by].dtypes != np.float64 and \
                                df[count_by].dtypes != np.float32

def colorRenderOrder(df, color_by, count_by, count_by_set=False):
    return colorQuantities(df, color_by, count_by, count_by_set).sort_values(ascending=False)

def colorQuantities(df, color_by, count_by, count_by_set=False):
    # Make sure we can count by numeric summation
    if count_by_set == False:
        count_by_set = countBySet(df, count_by)

    # For count by set... when count_by == color by... then we'll count by rows
    if count_by is not None and count_by_set and count_by == color_by:
        count_by = None

    if count_by is None:
        return df.groupby(color_by).size()
    elif count_by_set:
        _df = pd.DataFrame(df.groupby([color_by,count_by]).size()).reset_index()
        return _df.groupby(color_by).size()
    elif count_by == color_by:
        _df = df.groupby(color_by).size().reset_index()
        _df['__mult__'] = _df.apply(lambda x: x[color_by]*x[0],axis=1)
        return _df.groupby(color_by)['__mult__'].sum()
    else:
        return df.groupby(color_by)[count_by].sum()

for count_by in _combos_:
    for color_by in _combos_:
        if color_by is None:
            continue
        for _t_or_f_ in range(0,2):
            if _t_or_f_ == 0:
                count_by_set = False
            else:
                count_by_set = True
            _quants = colorQuantities(df, color_by, count_by, count_by_set)
            _order  = colorRenderOrder(df, color_by, count_by, count_by_set)


In [ ]:
#
# Examples for each case of count_by, color_by
#

# Count By Rows
color_by     = 'f'
count_by     = None
count_by_set = False
sorted(df.groupby(color_by).size(),reverse=True)

# Count By Set
color_by = 'f'
count_by = 'd'
count_by_set = True
_df = pd.DataFrame(df.groupby([color_by,count_by]).size()).reset_index()
sorted(_df.groupby(color_by).size(),reverse=True)

# Sum By Numbers
color_by = 'c'
count_by = 'f'
count_by_set = False
sorted(df.groupby(color_by)[count_by].sum(),reverse=True)

# count_by = color_by , Sum By Numbers
color_by = count_by = 'f'
count_by_set = False
_df = df.groupby(color_by).size().reset_index()
_df['__mult__'] = _df.apply(lambda x: x[color_by]*x[0],axis=1)
sorted(_df.groupby(color_by)['__mult__'].sum(),reverse=True)

# count_by = color_by , Count By Set // In this case, let's make count_by into rows...
color_by = count_by = 'f'
count_by_set = True

In [ ]:
bar_sz  = 10    # bar width  (assuming vertical bars)
bar_len = 100   # bar height (assumting vertical bars)

h_svg = f'<svg width="{bar_len+20}" height="{bar_sz*len(_combos_)*len(_combos_)}">'
v_svg = f'<svg width="{bar_sz*len(_combos_)*len(_combos_)}" height="{bar_len+20}">'

d = 0
for count_by in _combos_:
    for color_by in _combos_:
        count_by_set = False
        if count_by != None and df[count_by].dtypes != np.int64   and \
                                df[count_by].dtypes != np.int32   and \
                                df[count_by].dtypes != np.float64 and \
                                df[count_by].dtypes != np.float32:
            count_by_set = True
        global_color_order = rt.colorRenderOrder(df, color_by, count_by, count_by_set)
        h_svg += rt.colorizeBar(df, global_color_order, color_by, count_by, count_by_set,
                                10, d,          bar_len, bar_sz, True)
        v_svg += rt.colorizeBar(df, global_color_order, color_by, count_by, count_by_set,
                                d,  10+bar_len, bar_len, bar_sz, False)

        d += bar_sz

h_svg += '</svg>'
v_svg += '</svg>'

In [ ]:
rt.displaySVG(h_svg)

In [ ]:
rt.displaySVG(v_svg)